<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/RNN/RNN_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import shutil
import tensorflow as tf

from sklearn.metrics import accuracy_score, confusion_matrix

In [6]:
def download_and_read(url):
  local_file = url.split('/')[-1]
  local_file = local_file.replace("%20", " ")
  p = tf.keras.utils.get_file(local_file, url, extract=True, cache_dir=".")
  local_folder = os.path.join("drive/MyDrive/Datasets/UCI_ML_ES", local_file.split('.')[0])
  labeled_sentences = []
  for labeled_filename in os.listdir(local_folder):
    if labeled_filename.endswith("_labelled.txt"):
      with open(os.path.join(local_folder, labeled_filename), "r") as f:
        for line in f:
          sentence, label = line.strip().split("\t")
          labeled_sentences.append((sentence, label))

  return labeled_sentences
labeled_sentences = download_and_read(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip"
)
sentences = [s for (s, l) in labeled_sentences]
labels = [l for (s, l) in labeled_sentences]

In [14]:
#문장의 각 단어를  token으로 변환하기
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_counts)
print("vocabulary size : {:d}".format(vocab_size))

word2idx = tokenizer.word_index
idx2word = {v:k for (k, v) in word2idx.items()}

vocabulary size : 5271


In [16]:
#정적인 입력크기를 정하기 위해, 대부분 문장을 수용할 수 있을 만큼의 시퀀스 길이를 정한다.
seq_lengths = np.array([len(s.split()) for s in sentences])
print([(p, np.percentile(seq_lengths, p)) for p in [75, 80, 90, 95, 99, 100]])
#75% : 16단어 , 80% 18단어, 99% 36단어, 최대길이 71단어.

[(75, 16.0), (80, 18.0), (90, 22.0), (95, 26.0), (99, 36.0), (100, 71.0)]


In [ ]:
#최대 단어길이 64로 설정
max_seqlen = 64

#데이터 셋 생성
sentences_as_ints = tokenizer.texts_to_sequences(sentences)
#남은 부분은 0으로 채워준다.
sentences_as_ints = tf.keras.preprocessing.sequence.pad_sequences(sentences_as_ints, maxlen=max_seqlen)
labels_as_ints = np.array(labels)
dataset = tf.data.Dataset.from_tensor_slices((sentences_as_ints, labels_as_ints))

dataset = dataset.suffle(10000)
test_size = len(sentences) // 3
val_size = (len(sentences) - test_size) // 10
test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)

batch_size = 64
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)